In [75]:
import pandas as pd
import numpy as np
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body_barycentric, get_body, get_moon
from astropy.time import Time

def transform(x):
    return float(x.split(' ')[0])

transform = np.vectorize(transform)

df = pd.read_csv('./input/helio_data.csv')

pos = np.array([])
vel = np.array([])

pos = df.iloc[:, 2:5].values
pos = transform(pos)
pos = pos*1000

vel = df.iloc[:, 5:8].values
vel = transform(vel)
vel = vel*1000


In [113]:
# t = Time("2014-09-22 23:22")
# loc = EarthLocation.of_site('greenwich') 
# with solar_system_ephemeris.set('builtin'):
# mars = get_body('mars', t, loc)

def get_mars_pos(t):
    arg = 86400
    tini = 1515110400
    stamp = tini + t * arg
    #print(stamp, "from mars pos")
    tym = Time(stamp, format='unix')
    v = np.array([])
    with solar_system_ephemeris.set('de432s'):
        mars = get_body_barycentric('mars', tym)
        x = float(str(mars.x).split(' ')[0])
        y = float(str(mars.y).split(' ')[0])
        z = float(str(mars.z).split(' ')[0])
        v = np.append(v, [x, y, z])
#       print(v)
    return v
        

In [114]:
M1 = 1.989E30
M2 = 1000
G = 6.674E-11

def RK4(h, t_ini, r_ini, v_ini, t_final):
    # tfinal is days
    
    n = int((t_final - t_ini)/h)

    r = r_ini
    v = v_ini
    
    del_r = 0
    del_v = 0
    
    ast_distance = list()
    r2 = M1*r/(M1+M2)
#   err = list()
    print(n, "\niterations\n")
    for i in range(0, n+1):
        #print(i, r, v)
        
        t = (t_ini + i*h)*86400
        
        mars_pos = get_mars_pos(i*h)
        
        r += del_r
        v += del_v
        
        k1 = h*f(t,r)
        k2 = h*f(t+h/2, r+(h/2.0)*v+(h/8.0)*k1)
        k3 = h*f(t+h, r+h*v+(h/2.0)*k2)
        
        del_r = h*(v + (k1+2*k2)/6.0)
        del_v = (k1 +4*k2+k3)/6.0
        
        r2 = M1*r/(M1+M2)
        r23 = r2 - mars_pos
        #print(r23)
        ast_distance.append(np.linalg.norm(r23))
        
#         e = get_energy(r, v)
#         rel_err = (e - e0)/e0
#         err.append(rel_err)
        
#     niter = np.linspace(0, n, n+1)
#     fig = plt.figure()
#     fig.set_size_inches(8, 6)
#     ax = fig.add_subplot(211)
#     plt.tight_layout()
#     #ax.set_ylim(bottom=0.0)
    
#     # FROM POV OF PARTICLE 1
#     ax.plot(coordx, coordy, "y-")
    
#     # FROM POV OF CENTER OF MASS
#     ax.plot(r1x, r1y, "b-")
#     ax.plot(r2x, r2y, "r-")
    
#     ax.set_title('Initial orbit')
#     ax.set_aspect('equal', adjustable='box')
# #     ax.set_xlim([-0.75, 1.0])
# #     ax.set_ylim([-1.0, 1.0])
#     ax.set_xlabel('horizontal in arcsec')
#     ax.set_ylabel('vertical in arcsec')
#     ax.text(0.0, 0.0, 'o', color='green', fontsize=13)
    
#     ax2 = fig.add_subplot(212)
#     plt.tight_layout()
#     ax2.plot(niter, err)
#     ax2.set_xlabel('number of iterations')
#     ax2.set_ylabel('relative error in energy')
#     plt.show()
    return ast_distance

In [115]:
def f(t,r):
    y = -(G*(M1+M2)/np.power((np.linalg.norm(r)), 3))*r
    return y

In [130]:
r_ini = np.array([1.0, 0.0, 0.0])
v_ini = np.array([0.0, 0.75, 0.0])
t_ini = 0
t_final = 1000
h = 1
t_orbit = (4.0/7.0)**1.5 * 2.0 * np.pi

days_array_per_asteroid = list()

n_asteroids = 2

# Orbit time valid for M!+M2 = 1 onlu
# High accuracy will be achieved for position at final time = position at initial time. 
# i.e. integral number of orbits 
# t = 19 in this case
# print("First few orbit times")
# for i in range (1, 10):
#     print(i*t_orbit)
#print(np.power(np.linalg.norm(r_ini), 3))

for i in range(1, n_asteroids):
    min_dist = float(1.0E45)
    min_index = 0
    r_ini = pos[i,:]
    v_ini = vel[i,:]
    
    output = RK4(h, t_ini, r_ini, v_ini, t_final)
    print(t_final)
    for j in range(0, t_final):
        if(output[j] < min_dist):
            min_dist = output[j]
            min_index = j
    print(min_dist, min_index)
#     for vector in output:
    
#     days_array_per_asteroid.append(output)

    print(len(output))
    

1000 
iterations

1000
245336004257.0 0
1001
